In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey




In [2]:
fire_df = pd.read_csv ('Building_Fire.csv')
#branch
fire_df 

,ID,INCIDENT,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,MAPSCO,COUNCIL_DISTRICT,CENSUS_TRACT,STRUCTURE_TYPE,NO_OF_STORIES,CONSTRUCTION,CAUSE,TOTAL_VALUE,STRUCTURE_LOSS,CONTENTS_LOSS,TOTAL_SAVED,SPRINKER_SYS,ALARM_SYS
0,17961.0,2.019261e+09,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,55J,4.0,5400.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN
1,17962.0,2.019261e+09,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,23H,13.0,9702.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN
2,17963.0,2.019261e+09,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,64R,3.0,11200.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN
3,17964.0,2.019261e+09,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,16X,10.0,7804.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN
4,17965.0,2.019261e+09,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,42U,6.0,10602.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,18560.0,2.020237e+09,10/20/2019,06:11 PM,1209 ROUND TABLE DR,75247.0,9/A,33N,6.0,10000.0,PALLET OF STONE,NaN,NaN,INCENDIARY,838940.0,0.0,-10000.0,828940.0,NaN,NO
1030,NaN,NaN,02/06/2020,03:27 PM,2325 HILLGLENN RD,75228.0,4/B-E,38R-E,NaN,NaN,FENCE,NaN,WOOD,EXPOSURE (UNDETERMINED),10000.0,-500.0,0.0,9500.0,NaN,UNK
1031,NaN,2.021025e+09,02/01/2020,04:29 PM,1542 FORDHAM RD,75216.0,5/C,65B,4.0,5700.0,DWELLING,1.0,WOOD,UNDETERMINED,171340.0,-2000.0,-40.0,169300.0,NO,NO
1032,NaN,2.021025e+09,02/01/2020,02:32 PM,5500 JULIUS SCHEPPS FWY NB,75216.0,1/C,56B,7.0,8603.0,VEHICLE,2011.0,NISSAN ALTIMA,INCENDIARY (ATTEMPTED SUICIDE),5200.0,-4000.0,0.0,1200.0,NaN,NO


In [3]:
df = fire_df.drop("MAPSCO", axis='columns')
new_fire_df = df.drop("CENSUS_TRACT", axis='columns')
newest_fire_df = new_fire_df.drop("INCIDENT", axis='columns')
newest_fire_df

,ID,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL_DISTRICT,STRUCTURE_TYPE,NO_OF_STORIES,CONSTRUCTION,CAUSE,TOTAL_VALUE,STRUCTURE_LOSS,CONTENTS_LOSS,TOTAL_SAVED,SPRINKER_SYS,ALARM_SYS
0,17961.0,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN
1,17962.0,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN
2,17963.0,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN
3,17964.0,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN
4,17965.0,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,18560.0,10/20/2019,06:11 PM,1209 ROUND TABLE DR,75247.0,9/A,6.0,PALLET OF STONE,NaN,NaN,INCENDIARY,838940.0,0.0,-10000.0,828940.0,NaN,NO
1030,NaN,02/06/2020,03:27 PM,2325 HILLGLENN RD,75228.0,4/B-E,NaN,FENCE,NaN,WOOD,EXPOSURE (UNDETERMINED),10000.0,-500.0,0.0,9500.0,NaN,UNK
1031,NaN,02/01/2020,04:29 PM,1542 FORDHAM RD,75216.0,5/C,4.0,DWELLING,1.0,WOOD,UNDETERMINED,171340.0,-2000.0,-40.0,169300.0,NO,NO
1032,NaN,02/01/2020,02:32 PM,5500 JULIUS SCHEPPS FWY NB,75216.0,1/C,7.0,VEHICLE,2011.0,NISSAN ALTIMA,INCENDIARY (ATTEMPTED SUICIDE),5200.0,-4000.0,0.0,1200.0,NaN,NO


In [4]:
newest_fire_df[['BATTALION', 'SHIFT']] = newest_fire_df['BATTALION/SHIFT'].str.split('/', expand=True)
newest_fire_df

,ID,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL_DISTRICT,STRUCTURE_TYPE,NO_OF_STORIES,CONSTRUCTION,CAUSE,TOTAL_VALUE,STRUCTURE_LOSS,CONTENTS_LOSS,TOTAL_SAVED,SPRINKER_SYS,ALARM_SYS,BATTALION,SHIFT
0,17961.0,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN,6,A
1,17962.0,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN,7,A
2,17963.0,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN,5,B
3,17964.0,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN,2,B
4,17965.0,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN,9,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,18560.0,10/20/2019,06:11 PM,1209 ROUND TABLE DR,75247.0,9/A,6.0,PALLET OF STONE,NaN,NaN,INCENDIARY,838940.0,0.0,-10000.0,828940.0,NaN,NO,9,A
1030,NaN,02/06/2020,03:27 PM,2325 HILLGLENN RD,75228.0,4/B-E,NaN,FENCE,NaN,WOOD,EXPOSURE (UNDETERMINED),10000.0,-500.0,0.0,9500.0,NaN,UNK,4,B-E
1031,NaN,02/01/2020,04:29 PM,1542 FORDHAM RD,75216.0,5/C,4.0,DWELLING,1.0,WOOD,UNDETERMINED,171340.0,-2000.0,-40.0,169300.0,NO,NO,5,C
1032,NaN,02/01/2020,02:32 PM,5500 JULIUS SCHEPPS FWY NB,75216.0,1/C,7.0,VEHICLE,2011.0,NISSAN ALTIMA,INCENDIARY (ATTEMPTED SUICIDE),5200.0,-4000.0,0.0,1200.0,NaN,NO,1,C


In [5]:
newest_fire_df.rename(columns = {'COUNCIL_DISTRICT':'COUNCIL DISTRICT', 
                                 'STRUCTURE_TYPE':'STRUCTURE TYPE', 
                                 'NO_OF_STORIES':'BUILDING STORIES', 
                                 'TOTAL_VALUE':"TOTAL VALUE",
                                 'STRUCTURE_LOSS':'STRUCTURE LOSS',
                                 'CONTENTS_LOSS':'CONTENTS LOSS',
                                 'TOTAL_SAVED':'TOTAL SAVED', 
                                 'SPRINKER_SYS':'SPRINKER SYSTEM', 
                                 'ALARM_SYS':"ALARM SYSTEM",
                                }, inplace = True)
newest_fire_df.head()

,ID,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,CONSTRUCTION,CAUSE,TOTAL VALUE,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,SPRINKER SYSTEM,ALARM SYSTEM,BATTALION,SHIFT
0,17961.0,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN,6,A
1,17962.0,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN,7,A
2,17963.0,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN,5,B
3,17964.0,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN,2,B
4,17965.0,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN,9,B


In [6]:
'''
drop columns:
MAPSCO
Census Tract
drop rows:
stories < 100
total_saved > 0
total_value > 1


There are 621 rows

'''

clean_fire_df = newest_fire_df.loc[(newest_fire_df["BUILDING STORIES"] < 100) & (newest_fire_df["TOTAL SAVED"]> 0) \
                                & (newest_fire_df["TOTAL VALUE"] > 1 )]
clean_fire_df.head()

,ID,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,CONSTRUCTION,CAUSE,TOTAL VALUE,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,SPRINKER SYSTEM,ALARM SYSTEM,BATTALION,SHIFT
0,17961.0,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN,6,A
1,17962.0,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN,7,A
2,17963.0,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN,5,B
3,17964.0,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN,2,B
4,17965.0,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN,9,B


In [7]:
#clean_fire_df.loc[(clean_fire_df["TOTAL_SAVED"] > 0)]

total_save = clean_fire_df["TOTAL SAVED"]
total_value = clean_fire_df["TOTAL VALUE"]

percentage_saved = total_save / total_value * 100

percentage_loss = (100 - percentage_saved).round(2)


percentage_loss

clean_fire_df["PERCENTAGE LOSS"] = percentage_loss

clean_fire_df

/var/folders/qt/z1139kj16rq91qs03f19sz2c0000gn/T/ipykernel_63953/1259834470.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fire_df["PERCENTAGE LOSS"] = percentage_loss


,ID,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,CONSTRUCTION,CAUSE,TOTAL VALUE,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,SPRINKER SYSTEM,ALARM SYSTEM,BATTALION,SHIFT,PERCENTAGE LOSS
0,17961.0,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN,6,A,3.01
1,17962.0,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN,7,A,1.87
2,17963.0,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN,5,B,6.54
3,17964.0,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN,2,B,9.47
4,17965.0,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN,9,B,65.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,18414.0,09/03/2020,02:12 AM,5024 URBAN AVE,75227.0,8/A,7.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,261000.0,-15000.0,-2500.0,243500.0,NO,YES,8,A,6.70
1021,18459.0,09/19/2020,08:49 AM,5307 GASTON AVE,75214.0,3/C,14.0,APARTMENTS,2.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,2240263.0,-2500.0,0.0,2237763.0,NaN,UNK,3,C,0.11
1025,18496.0,09/24/2020,07:11 PM,218 W FERRIS BRANCH BLVD,75243.0,4/B,10.0,APARTMENTS,3.0,WOOD,EQUIPMENT UNATTENDED,7357830.0,-200.0,0.0,7357630.0,YES,UKN,4,B,0.00
1028,18514.0,09/30/2020,07:15 AM,14770 LASATER RD,75253.0,8/B,8.0,SHED,1.0,METAL,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,1500.0,-500.0,-100.0,900.0,NO,NO,8,B,40.00


In [13]:
clean_fire_df = clean_fire_df[["ID", "DATE", "TIME", "ZIP", "BATTALION", "SHIFT", 
                "COUNCIL DISTRICT", "STRUCTURE TYPE", "BUILDING STORIES",
                "CONSTRUCTION", "CAUSE", "TOTAL VALUE", "STRUCTURE LOSS",
                "CONTENTS LOSS", "TOTAL SAVED", "PERCENTAGE LOSS", "SPRINKER SYSTEM",
                "ALARM SYSTEM", "ADDRESS"]]
clean_fire_df

,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,CONSTRUCTION,CAUSE,TOTAL VALUE,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,PERCENTAGE LOSS,SPRINKER SYSTEM,ALARM SYSTEM,ADDRESS
0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE
1,17962.0,10/30/2019,03:42 AM,75229.0,7,A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,1.87,NaN,NaN,10728 LATHROP DR
2,17963.0,10/30/2019,10:47 AM,75241.0,5,B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,6.54,NaN,NaN,6205 S R L THORNTON SERV
3,17964.0,10/30/2019,05:32 PM,75243.0,2,B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,9.47,NaN,NaN,12044 WILLOWDELL DR
4,17965.0,10/30/2019,11:03 AM,75212.0,9,B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,65.48,NaN,NaN,2212 LAPSLEY ST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,18414.0,09/03/2020,02:12 AM,75227.0,8,A,7.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,261000.0,-15000.0,-2500.0,243500.0,6.70,NO,YES,5024 URBAN AVE
617,18459.0,09/19/2020,08:49 AM,75214.0,3,C,14.0,APARTMENTS,2.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,2240263.0,-2500.0,0.0,2237763.0,0.11,NaN,UNK,5307 GASTON AVE
618,18496.0,09/24/2020,07:11 PM,75243.0,4,B,10.0,APARTMENTS,3.0,WOOD,EQUIPMENT UNATTENDED,7357830.0,-200.0,0.0,7357630.0,0.00,YES,UKN,218 W FERRIS BRANCH BLVD
619,18514.0,09/30/2020,07:15 AM,75253.0,8,B,8.0,SHED,1.0,METAL,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,1500.0,-500.0,-100.0,900.0,40.00,NO,NO,14770 LASATER RD


In [14]:
clean_fire_df = clean_fire_df.reset_index()

clean_fire_df

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,CONSTRUCTION,CAUSE,TOTAL VALUE,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,PERCENTAGE LOSS,SPRINKER SYSTEM,ALARM SYSTEM,ADDRESS
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE
1,1,17962.0,10/30/2019,03:42 AM,75229.0,7,A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,1.87,NaN,NaN,10728 LATHROP DR
2,2,17963.0,10/30/2019,10:47 AM,75241.0,5,B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,6.54,NaN,NaN,6205 S R L THORNTON SERV
3,3,17964.0,10/30/2019,05:32 PM,75243.0,2,B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,9.47,NaN,NaN,12044 WILLOWDELL DR
4,4,17965.0,10/30/2019,11:03 AM,75212.0,9,B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,65.48,NaN,NaN,2212 LAPSLEY ST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,616,18414.0,09/03/2020,02:12 AM,75227.0,8,A,7.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,261000.0,-15000.0,-2500.0,243500.0,6.70,NO,YES,5024 URBAN AVE
617,617,18459.0,09/19/2020,08:49 AM,75214.0,3,C,14.0,APARTMENTS,2.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,2240263.0,-2500.0,0.0,2237763.0,0.11,NaN,UNK,5307 GASTON AVE
618,618,18496.0,09/24/2020,07:11 PM,75243.0,4,B,10.0,APARTMENTS,3.0,WOOD,EQUIPMENT UNATTENDED,7357830.0,-200.0,0.0,7357630.0,0.00,YES,UKN,218 W FERRIS BRANCH BLVD
619,619,18514.0,09/30/2020,07:15 AM,75253.0,8,B,8.0,SHED,1.0,METAL,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,1500.0,-500.0,-100.0,900.0,40.00,NO,NO,14770 LASATER RD


In [22]:
from IPython.display import clear_output

lats = []
lngs = []
# geocoordinates
places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
geocode_url = "https://maps.googleapis.com/maps/api/geocode/json"
for index, row in clean_fire_df.iterrows():
    address = row['ADDRESS'] + ", Dallas, TX"
       
    
    params = {
        "input": address.replace(" ", "+"),
        "inputtype": "textquery",
        "key": gkey,
        "fields": "geometry"
    }
    
    
        
    try:
        geo_data = requests.get(places_url, params=params).json()
        lat = geo_data["candidates"][0]["geometry"]["location"]["lat"]
        lng = geo_data["candidates"][0]["geometry"]["location"]["lng"]
        
    except:
        lat = None
        lng = None

    clear_output(wait=True)
    print(f"{index}: {lat}, {lng}")

    
    lats.append(lat)
    lngs.append(lng)

        
    
clean_fire_df['Lat'] = lats
clean_fire_df['Lng'] = lngs


620: 32.696174, -96.8020909


In [21]:
lats

[32.7321074,
 32.7321074,
 32.8942481,
 32.8942481,
 32.6945433,
 32.6945433,
 32.9135841,
 32.9135841,
 32.7699863,
 32.7699863,
 32.7902897,
 32.7902897,
 32.713743,
 32.713743,
 32.7877762,
 32.7877762,
 32.6768475,
 32.6768475,
 32.7981654,
 32.7981654,
 32.7533216,
 32.7533216,
 32.770471,
 32.770471,
 32.7850416,
 32.7850416,
 32.7199652,
 32.7199652,
 32.7805512,
 32.7805512,
 32.8369595,
 32.8369595,
 32.780645,
 32.780645,
 32.7380103,
 32.7380103,
 32.914552,
 32.914552,
 32.7396016,
 32.7396016,
 32.7929463,
 32.7929463,
 32.737115,
 32.737115,
 32.63636880000001,
 32.63636880000001,
 32.880411,
 32.880411,
 32.8151764,
 32.8151764,
 32.822634,
 32.822634,
 32.782672,
 32.782672,
 32.80619800000001,
 32.80619800000001,
 32.7521136,
 32.7521136,
 32.935232,
 32.935232,
 32.8207189,
 32.8207189,
 32.7642823,
 32.7642823,
 32.9019505,
 32.9019505,
 32.7431175,
 32.7431175,
 32.7926156,
 32.7926156,
 32.7142542,
 32.7142542,
 32.6591207,
 32.6591207,
 32.8255109,
 32.8255109,
 3

In [26]:
(len(lats))



621

In [1]:
clean_fire_df.column

NameError: name 'clean_fire_df' is not defined

In [25]:
clean_fire_df.to_csv(r'Fire.csv', index = False)
